(mspsc_pt2)=
# Miniature/spontaneous postsynaptic currents: Part 2

In part 1 we covered how to extract the mEPSCs or sEPSCs. In this chapter we will dive into a more rigorous analysis of PSCs as well as look at how the shape of a PSC effects their integration.

In [1]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Select
from bokeh.plotting import figure
from scipy import stats

output_notebook()

Loading BokehJS ...

In [2]:
url_path = "https://cdn.jsdelivr.net/gh/LarsHenrikNelson/PatchClampHandbook/data/"
pv = pd.read_csv(url_path + "pv/mini_data.csv")
msn = pd.read_csv(url_path + "msn/mini_data.csv")
dfs = {"pv": pv, "msn":msn}

In [6]:
columns = [
    "Est Tau (ms)",
    "Rise Time (ms)",
    "Amplitude (pA)",
    "Rise Rate (pA/ms)",
    "IEI (ms)",
]
def kde(data):
    min_val = min(data)
    max_val = max(data)
    padding = (max_val - min_val) * 0.1  # Add 10% padding
    grid_min = min_val - padding
    grid_max = max_val + padding
    grid_min = max(grid_min, 0)
    positions = np.linspace(grid_min, grid_max, num=124)
    kernel = stats.gaussian_kde(data)
    y = kernel(positions)
    return positions, y

source_dict = {key: {} for key in dfs.keys()}
for key, dataset in dfs.items():
    for i in columns:
        data = dataset[i].dropna()
        data = data[data != 0]
        positions, y = kde(data)
        source_dict[key][f"{i}_x_identity"] = positions
        source_dict[key][f"{i}_y_identity"] = y
        positions, y = kde(np.log10(data))
        source_dict[key][f"{i}_x_log"] = positions
        source_dict[key][f"{i}_y_log"] = y

    source_dict[key]["y"] = source_dict[key]["Amplitude (pA)_y_identity"]
    source_dict[key]["x"] = source_dict[key]["Amplitude (pA)_x_identity"]
    source_dict[key] = ColumnDataSource(source_dict[key])

## Relationships between variables

In [8]:
figs = []
for key, value in source_dict.items():
    fig = figure(height=250, width=350)
    line = fig.line(x="x", y="y", source=value, line_color="black")
    figs.append(fig)

menu = Select(title="Variables", value=columns[0], options=columns)
transform = Select(
    title="Transform",
    value="identity",
    options=["identity", "log"],
)

callback = CustomJS(
    args=dict(
        source=source_dict,
        keys=list(source_dict.keys()),
        menu=menu,
        transform=transform
    ),
    code="""
    for (const item of keys) {
        source[item].data.y = source[item].data[`${menu.value}_y_${transform.value}`];
        source[item].data.x = source[item].data[`${menu.value}_x_${transform.value}`];
        source[item].change.emit();
    };
""",
)

menu.js_on_change("value", callback)
transform.js_on_change("value", callback)

show(column(row(menu, transform), row(figs)))